In [ ]:
#installations
!pip install kshingle

In [ ]:
!pip install prince

In [ ]:
!pip install rectangle-packer

In [ ]:
!pip install grandalf

In [1]:
#from d3m import container
import datamart
import datamart_rest
import datamart_profiler
from datamart_profiler import temporal
import pandas as pd

from io import BytesIO
import requests
import json
import math
import csv
from datetime import datetime
from dateutil.parser import parse

import numpy as np
from datetime import datetime
import kshingle as ks
import random
import itertools
import prince
import warnings
from functools import wraps
from itertools import combinations, product
from scipy.spatial import Voronoi
from scipy.spatial.distance import cdist, pdist, squareform
from scipy.optimize import minimize, NonlinearConstraint
from rpack import pack
from grandalf.graphs import Vertex, Edge, Graph
from grandalf.layouts import SugiyamaLayout, DummyVertex
from netgraph_functions import get_geometric_layout, _initialise_geometric_node_layout, _flatten, _get_unique_nodes, get_fruchterman_reingold_layout, _edge_list_to_adjacency_matrix, _get_fr_repulsion, _get_fr_attraction, _fruchterman_reingold, _get_temperature_decay, _is_within_bbox, _fit_to_frame, _get_angle, _rotate
from custom_functions import jaccard_similarity, distance_from_similarity, get_edges, get_edge_lengths, normalize, get_node_positions, get_df_cols

In [2]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("Python Spark SQL basic example") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/04 14:05:47 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/04 14:05:47 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/04 14:05:47 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/04 14:05:48 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


## Generate Metadata
The first step in this notebook is to conduct a keyword search against the Auctus REST API. It then constructs a CSV based on the results of that search.

We search for datasets in Auctus using the REST API. The official documentation for the Auctus API can be found here: https://docs.auctus.vida-nyu.org/rest/

In [4]:
def print_results(results):
    if not results:
        return
    for idx, result in enumerate(results):
        print("Result INDEX: " + str(idx))
        print("ID: " + result['id'])
        print("Name: " + result['metadata']['name'])
        print("Size: " + str(result['metadata']['size']))
        print('Score: ', result['score'])
        print("-------------------\n")

Changing the keyword below will change the search.

In [5]:
search_keywords = ['taxi']

We search for datasets in Auctus using the REST API. The official documentation can be found here: https://docs.auctus.vida-nyu.org/rest/

In [6]:
url = 'https://auctus.vida-nyu.org/api/v1/search'
datasets_per_page = 50

query = {
    'keywords': search_keywords
}
response = requests.post(
        url,
        files={
            'query': ('query.json', json.dumps(query), 'application/json'),
        }
    )

In [7]:
response.raise_for_status()
query_results = response.json()['results']
total_datasets = response.json()['total']
num_pages = math.ceil(total_datasets / datasets_per_page)
print(num_pages)

3


By default, the Auctus API only returns only the first 50 results; we make additional calls to retrieve the full results; since we already have the first page, we start on the second page.

In [8]:
# By default, the Auctus API returns only the first 50 results; we make additional calls to retrieve
# the full results; since we already have the first page, we start on the second page
all_results = query_results
for p in range(num_pages - 1):
    url = 'https://auctus.vida-nyu.org/api/v1/search?page=' + str(p + 2)
    print(url)

    query = {
        'keywords': search_keywords
    }
    response = requests.post(
            url,
            files={
                'query': ('query.json', json.dumps(query), 'application/json'),
            }
        )
    response.raise_for_status()
    query_results.extend(response.json()['results'])

https://auctus.vida-nyu.org/api/v1/search?page=2
https://auctus.vida-nyu.org/api/v1/search?page=3


In [9]:
print("Retrieved " + str(len(all_results)) + " results: ")
print("*****")
print_results(all_results)

Retrieved 109results: 
 ***** 
Result INDEX: 0
datamart.socrata.data-cityofchicago-org.6u8z-aubf
Taxi Medallion Transfers
304895
Score:  23.785912
-------------------

Result INDEX: 1
datamart.upload.dd6b73540cff4fe59e71892e4a75047b
2017 Yellow Taxi Data
13949149
Score:  22.182976
-------------------

Result INDEX: 2
datamart.upload.d5cff1264b974b0bb3e6008c314fdf16
Green Taxi Data 2015
1066116963
Score:  22.182976
-------------------

Result INDEX: 3
datamart.upload.4eb2156e6a994f33ba71dd59f44c4c59
Yellow Taxi Data 2015
20244
Score:  22.182976
-------------------

Result INDEX: 4
datamart.url.54f251dd89075077b3b76edfb2fcbbdd
2017-yellow-cab-lga
319452961
Score:  22.007399
-------------------

Result INDEX: 5
datamart.socrata.data-cityofnewyork-us.2np7-5jsg
2014 Green Taxi Trip Data
2523253343
Score:  20.782446
-------------------

Result INDEX: 6
datamart.socrata.data-cityofnewyork-us.gi8d-wdg5
2015 Green Taxi Trip Data
3166006002
Score:  20.782446
-------------------

Result INDEX: 7


Before adding to them to the metadata, we standardize all of the column names. We remove any non-ascii characters and convert them to snake case by replacing all whitespace with an underscore.

In [10]:
import string, unicodedata, re
def standardize_col_name(col_name):
    name = unicodedata.normalize('NFKD', col_name).encode('ascii','ignore').decode()
    name = re.sub('\s+', '_', name)
    return name.lower()

We import and use the datatypes defined by the Auctus profiler, which can be found in the following file: https://gitlab.com/ViDA-NYU/auctus/auctus/-/blob/master/lib_profiler/datamart_profiler/types.py 

Auctus itself conforms to the schema.org standards, whose documentation can be found here: https://schema.org/docs/schemas.html

In [11]:
from datamart_profiler import types

Here, we iterate over the metadata that Auctus returned from our search. These results will be used to create the data for the dataset summarizer visualization.

We also pass in the full metadata to display alongside the similarity measurements, and today's date as a way of timestamping these results.

In [12]:
separator = ", "
metadatas = []
start_date = ''
end_date = ''
for idx, result in enumerate(query_results):
    metadata = result['metadata']
    dataset_id = query_results[idx]['id']
    
    temp_type_cols = [standardize_col_name(col['name']) for col in metadata['columns'] if types.DATE_TIME in col['semantic_types']]
    cat_type_cols = [standardize_col_name(col['name']) for col in metadata['columns'] if types.CATEGORICAL in col['semantic_types']]
    spatial_type_cols = [standardize_col_name(col['name']) for col in metadata['columns'] if \
                         types.GEO_POINT in col['semantic_types'] or \
                         types.GEO_POLYGON in col['semantic_types'] or \
                         types.LATITUDE in col['semantic_types'] or \
                         types.LONGITUDE in col['semantic_types'] \
                        ]
    all_cols = [standardize_col_name(col['name']) for col in metadata['columns']]
    
    if 'temporal_coverage' in metadata.keys():
        start_date = ''
        end_date = ''
        for temporal_col in metadata['temporal_coverage']:
            c_start_date = datetime.fromtimestamp(temporal_col['ranges'][0]['range']['gte'])
            c_end_date = datetime.fromtimestamp(temporal_col['ranges'][-1]['range']['gte'])
            if start_date == '' or c_start_date < start_date:
                start_date = c_start_date
            if end_date == '' or c_end_date > end_date:
                end_date = c_end_date
        
    data = {
        'title': metadata['name'],
        'description': metadata['description'],
        'size': metadata['size'],
        'num_spatial': metadata['nb_spatial_columns'] if 'nb_spatial_columns' in metadata else 0,
        'num_categorical': metadata['nb_categorical_columns'] if 'nb_categorical_columns' in metadata else 0,
        'num_temporal': metadata['nb_temporal_columns'] if 'nb_temporal_columns' in metadata else 0,
        'num_columns': metadata['nb_columns'],
        'start_date': start_date,
        'end_date': end_date,
        'temporal_col_names': separator.join(temp_type_cols),
        'cat_col_names': separator.join(cat_type_cols),
        'spatial_col_names': separator.join(spatial_type_cols),
        'all_col_names': separator.join(all_cols),
        'full_metadata': metadata,
        'date_of_search': datetime.now()
    }
    metadatas.append(data)

In [13]:
p_metadata_dataframe = pd.DataFrame(data = metadatas)
display(p_metadata_dataframe)

,title,description,size,num_spatial,num_categorical,num_temporal,num_columns,start_date,end_date,temporal_col_names,cat_col_names,spatial_col_names,all_col_names,full_metadata,date_of_search
0,Taxi Medallion Transfers,Operation of a taxi cab in Chicago requires a ...,304895,0,0,1,5,2007-10-10 00:00:00,2017-07-21 00:00:00,closing_date,,,"closing_date, public_vehicle_number, sale_pric...","{'name': 'Taxi Medallion Transfers', 'source':...",2023-05-04 14:05:57.777532
1,2017 Yellow Taxi Data,This dataset includes trip records from all tr...,13949149,0,0,1,3,2017-07-03 23:00:16,2017-11-04 03:58:56,tpep_pickup_datetime,tpep_pickup_datetime,,"tpep_pickup_datetime, pulocationid, n._trips",{'attribute_keywords': ['tpep_pickup_datetime'...,2023-05-04 14:05:57.777565
2,Green Taxi Data 2015,This dataset contains green taxi trip records ...,1066116963,4,1,2,23,2015-01-02 21:26:24,2015-03-23 08:42:40,"pickup_datetime, dropoff_datetime",store_and_fwd_flag,"pickup_longitude, pickup_latitude, dropoff_lon...","vendorid, pickup_datetime, dropoff_datetime, s...","{'attribute_keywords': ['VendorID', 'Vendor', ...",2023-05-04 14:05:57.777682
3,Yellow Taxi Data 2015,This dataset contains the daily number of yell...,20244,0,0,1,4,2015-01-07 00:00:00,2015-09-05 00:00:00,pickup_datetime,,,"pickup_datetime, n._trips, price, distance","{'attribute_keywords': ['pickup_datetime', 'pi...",2023-05-04 14:05:57.777710
4,2017-yellow-cab-lga,"new york, taxi, yellow cab, LaGuardia, LGA, ci...",319452961,0,1,2,18,2017-01-08 13:20:00,2017-09-04 00:10:40,"tpep_pickup_datetime, tpep_dropoff_datetime",store_and_fwd_flag,,"tpep_pickup_datetime, unnamed:_0, dolocationid...",{'attribute_keywords': ['tpep_pickup_datetime'...,2023-05-04 14:05:57.777785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Monthly Transportation Statistics,Monthly Transportation Statistics is a compila...,492318,0,0,1,136,1948-04-01 00:00:00,1999-05-01 00:00:00,date,,,"index, date, air_safety_-_general_aviation_fat...","{'name': 'Monthly Transportation Statistics', ...",2023-05-04 14:05:57.792827
105,2010 Census/ACS Detailed Block Group Data,detailed characteristics of people and housing...,506671,1,0,0,190,1948-04-01 00:00:00,1999-05-01 00:00:00,,county,,"county, census_tract, block_group, block_group...",{'name': '2010 Census/ACS Detailed Block Group...,2023-05-04 14:05:57.793680
106,2013-2017 American Community Survey Detailed C...,DETAILED CHARACTERISTICS OF PEOPLE AND HOUSING...,435842,0,1,0,211,1948-04-01 00:00:00,1999-05-01 00:00:00,,inside_kcmo_or_not,,"island, census_tract, inside_kcmo_or_not, east...",{'name': '2013-2017 American Community Survey ...,2023-05-04 14:05:57.794625
107,Parking - Edmonton Insight Community,This was one single topic among many as part o...,1085353,0,28,2,74,2016-02-09 13:34:56,2016-02-11 16:34:08,"response_date, completion_date",q4a_parking_it_is_easy_to_find_a_place_to_park...,,"response_date, completion_date, q1a_parking_un...","{'attribute_keywords': ['Response Date', 'Resp...",2023-05-04 14:05:57.795305


We save the metadata in a CSV file marked with today's date. Because the search results from Auctus may change over time, this provides a way to version this result. If necessary, this CSV can be used rather than re-generating the metadata from scratch.

In [17]:
todays_date_snakecase = datetime.today().strftime('%Y_%m_%d')
p_metadata_dataframe.to_csv("taxi_metadata_" + todays_date_snakecase + ".csv", index=False)

We also dump the list of dataset IDs to a file along with their last updated dates from Auctus. If the search results do change (for example, if new datasets are added), this list can also be used to re-generate the metadata results that we've provided a hard copy of.

We again add the date to the filename as a means of versioning and timestamping the results.

In [16]:
filename = 'dataset_ids_' + todays_date_snakecase + '.csv'

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    for dataset_metadata in query_results:
        writer.writerow([dataset_metadata['id'], dataset_metadata['metadata']['materialize']['date']])

file.close()

['datamart.socrata.data-cityofchicago-org.6u8z-aubf', 'datamart.upload.dd6b73540cff4fe59e71892e4a75047b', 'datamart.upload.d5cff1264b974b0bb3e6008c314fdf16', 'datamart.upload.4eb2156e6a994f33ba71dd59f44c4c59', 'datamart.url.54f251dd89075077b3b76edfb2fcbbdd', 'datamart.socrata.data-cityofnewyork-us.2np7-5jsg', 'datamart.socrata.data-cityofnewyork-us.gi8d-wdg5', 'datamart.socrata.data-edmonton-ca.9unw-vz23', 'datamart.socrata.data-cityofnewyork-us.ghpb-fpea', 'datamart.socrata.data-edmonton-ca.yepf-actv', 'datamart.socrata.data-cityofnewyork-us.5gj9-2kzx', 'datamart.socrata.data-cityofnewyork-us.pkmi-4kfn', 'datamart.socrata.data-cityofnewyork-us.hvrh-b6nb', 'datamart.socrata.data-cityofnewyork-us.w7fs-fd9i', 'datamart.socrata.data-cityofnewyork-us.kxp8-n2sj', 'datamart.socrata.data-cityofnewyork-us.m6nq-qud6', 'datamart.socrata.data-cityofnewyork-us.biws-g3hs', 'datamart.socrata.data-cityofnewyork-us.t29m-gskq', 'datamart.socrata.data-cityofnewyork-us.q5mz-t52e', 'datamart.socrata.data-

23/05/04 14:11:35 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 2 for reason Container marked as failed: container_e07_1683077339950_1650_01_000010 on host: nyu-dataproc-sw-2qrr.c.hpc-dataproc-19b8.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
23/05/04 14:11:35 ERROR org.apache.spark.scheduler.cluster.YarnScheduler: Lost executor 2 on nyu-dataproc-sw-2qrr.c.hpc-dataproc-19b8.internal: Container marked as failed: container_e07_1683077339950_1650_01_000010 on host: nyu-dataproc-sw-2qrr.c.hpc-dataproc-19b8.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.


## Compute Similarity

We can now calculate the similarity between each pair of datasets based on various attributes. For each of these attributes, we create a set of k-shingles for each dataset and compute the jaccard similarity between each pair of k-shingle sets.

The function jaccard_similarity accepts three arguments, and can be used to calculate the similarity between pairs of datasets using any column containing strings from the metadata dataframe. The first argument is the metadata dataframe. The second argument is a list of the dataset attributes (or metadata dataframe columns) that we will compare each dataset by. These attributes will be concatenated into a single string for each dataset before similarity is calculated. The third argument is k value we will use to create the k-shingle sets for each dataset.  

We will calculate the similarity between each pair of datasets based on their titles, descriptions, and a string containing both their title and description. We choose to break the titles into sets of 5-shingles (since they are relatively short) and the descriptions and title/description strings into sets of 9-shingles (since these are relatively long).  

In [ ]:
df = p_metadata_dataframe
# get similarity matrices
matrix_title_sim = jaccard_similarity(df, ["title"], 5) 
matrix_desc_sim = jaccard_similarity(df, ["description"], 9) 
matrix_both_sim = jaccard_similarity(df, ["title","description"], 9)

Our final visualization will contain clusters of dots, each representing one dataset, with the distance between each pair of dots intuitively representing how similar they are. We want dots to be closer together if they are more similar and father apart if they are less similar, and therefore must convert this jaccard similarity to jaccard distance.

In [ ]:
#get distance matrices
matrix_title_dist = distance_from_similarity(matrix_title_sim)
matrix_desc_dist = distance_from_similarity(matrix_desc_sim)
matrix_both_dist = distance_from_similarity(matrix_both_sim)

Next we normalize these jaccard distances between the values 0.0001 and 1.0001. This will ensure that the matrices will not break the constraints of the least squares optimization we will perform using these values later (specifically, it will ensure that the matrices do not have a determinant of zero).  

In [ ]:
#normalize distances between 0.0001 and 1.0001 rather than 0 and 1 to avoid breaking least squares optimization constraints
matrix_title_dist_norm = normalize(matrix_title_dist, {'actual': {'lower': 0, 'upper': 1}, 'desired': {'lower': 0.0001, 'upper': 1.0001}})
matrix_desc_dist_norm = normalize(matrix_desc_dist, {'actual': {'lower': 0, 'upper': 1}, 'desired': {'lower': 0.0001, 'upper': 1.0001}})
matrix_both_dist_norm = normalize(matrix_both_dist, {'actual': {'lower': 0, 'upper': 1}, 'desired': {'lower': 0.0001, 'upper': 1.0001}})

We can now use the netgraph get_node_positions function to calculate the x and y coordinates of each dataset in the cluster based on the edge lengths (jaccard distances) between them. This function treats the cluster like a force directed graph and uses a sequential least squares programming optimizer to converge on a final layout. The nodes in this graph are datasets, and the length of an edge between two dataset nodes is equal to the jaccard distance between the datasets. These edges will not be visible in the final visualization.

In [ ]:
#get edges [tuples in the format (source node ID, target node ID)] and edge lengths [dictionary mapping edges to their distances] for each measure
edges = get_edges(df)

title_edge_lengths = get_edge_lengths(edges, matrix_title_dist_norm)
desc_edge_lengths = get_edge_lengths(edges, matrix_desc_dist_norm)
both_edge_lengths = get_edge_lengths(edges, matrix_both_dist_norm)

In [ ]:
#get x and y coordinates for each measure
title_node_positions = get_node_positions(edges, title_edge_lengths)
desc_node_positions = get_node_positions(edges, desc_edge_lengths)
both_node_positions = get_node_positions(edges, both_edge_lengths)

In [ ]:
print(title_node_positions[0:10]) #x column and y column 

Next we will use multiple correspondence analysis (MCA) to compute the similarity between the column names within each pair of datasets. Since MCA requires an input that is sorted into multiple categories, we will sort these column names  by whether they represent a "categorical," "spatial," "temporal," or "other" type of variable. 

In [ ]:
#separate column names by type
df_cols = get_df_cols(df)
display(df_cols)

We can now perform MCA on the dataset column names. 

In [ ]:
#perform MCA
X = df_cols
mca = prince.MCA()
mca = mca.fit(X)
mca = mca.transform(X)
display(mca)

Earlier we used the netgraph get_node_positions function to find the x and y position of each dataset within each similarity cluster. This function normalizes these positions between 0.05 and 0.95 so that the dots are framed nicely within the 1 by 1 background of the cluster visualization. We therefore normalize the x and y position of each dataset within the column name similarity cluster between 0.05 and 0.95 to match the netgraph padding. 

In [ ]:
#normalize values between 0.05 and 0.95
mca_arr = mca.to_numpy()
normalized_mca = np.array(normalize(mca_arr, {'actual': {'lower': mca.min(), 'upper': mca.max()}, 'desired': {'lower': 0.05, 'upper': 0.95}}))

Finally we add the positions of each dataset within each similarity cluster to the dataframe, rename the dataframe index to "id," and save it as a CSV.

In [ ]:
#append coordinates to dataframe
df["title_x"] = title_node_positions[:,0]
df["title_y"] = title_node_positions[:,1]

df["description_x"] = desc_node_positions[:,0]
df["description_y"] = desc_node_positions[:,1]

df["title_and_description_x"] = both_node_positions[:,0]
df["title_and_description_y"] = both_node_positions[:,1]

df["column_name_x"] = normalized_mca[:,0]
df["column_name_y"] = normalized_mca[:,1]

df.index.name = 'id'

In [ ]:
display(df)

In [ ]:
df.to_csv("/content/taxi_full_metadata_and_scatterplot_coordinates.csv")